In [23]:
import numpy as np
# load the packages needed
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from src.solver import Solver
from src.model import AR, MA
from src.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
import src.data_processor as dp
import src.inference 
from src.ts_gen import ar1_gen
data = np.loadtxt("../data/SP500array.csv", delimiter=',')

In [2]:
X=np.array([data[0:100,0]])
Y=dp.get_return(X)
X = ar1_gen(0.5, sigma = 1, time = 200, num = 1, burnin = 2000)

In [3]:
lag = 1
sigma = 1.0
intercept = 0.1
phi = np.random.randn(1, 1)
AR_model = AR(lag=lag, phi=phi, sigma=sigma, intercept=intercept)
AR_model.params

{'intercept': 0.1, 'phi': array([[ 1.11071189]]), 'sigma': 1.0}

In [4]:
_, grads = AR_model.loss(X)
solver = Solver(AR_model, X,
                          update_rule='sgd',
                          optim_config={
                            'learning_rate': 1e-1,
                          },
                          
                          num_epochs=1000, batch_size=1,
                          print_every=10)
solver.train()

1000
the loss is %f 615.179003938
the loss is %f 1121.88258826
the loss is %f 1474.10942806
the loss is %f 1474.01624738
the loss is %f 1473.92297996
the loss is %f 1473.82962565
the loss is %f 1473.73618428
the loss is %f 1473.64265569
the loss is %f 1473.54903972
the loss is %f 1473.4553362
the loss is %f 1473.36154497
the loss is %f 1473.26766586
the loss is %f 1473.17369871
the loss is %f 1473.07964335
the loss is %f 1472.98549962
the loss is %f 1472.89126735
the loss is %f 1472.79694638
the loss is %f 1472.70253653
the loss is %f 1472.60803763
the loss is %f 1472.51344953
the loss is %f 1472.41877205
the loss is %f 1472.32400502
the loss is %f 1472.22914827
the loss is %f 1472.13420163
the loss is %f 1472.03916493
the loss is %f 1471.944038
the loss is %f 1471.84882067
the loss is %f 1471.75351276
the loss is %f 1471.6581141
the loss is %f 1471.56262453
the loss is %f 1471.46704386
the loss is %f 1471.37137192
the loss is %f 1471.27560853
the loss is %f 1471.17975353
the loss is %

In [5]:
AR_model.params

{'intercept': -23.252554606847124,
 'phi': array([[-114.39287631]]),
 'sigma': -612.2193724737515}

In [53]:
AR_model.predict(X,10)

array([[ -2.25644028e+06,   5.40566422e+10,  -1.29498660e+15,
          3.10228351e+19,  -7.43186297e+23,   1.78038490e+28,
         -4.26510878e+32,   1.02175394e+37,  -2.44772447e+41,
          5.86379447e+45]])

In [18]:
from src.ts_gen import ar1_gen
import src.inference as inf

x = ar1_gen(0.5, sigma = 1, time = 200, num = 1, burnin = 2000)
X=np.array([data[0:100,0]])
Y=dp.get_return(X)

y = inf.yule_walker(x, order =1, method = "mle")
phi,sigma = inf.yule_walker(Y, order =2, method = "mle")
print phi 
print sigma



[-0.19786273 -0.05678889]
0.000119785804031


In [19]:

phi

array([-0.19786273, -0.05678889])

In [20]:
AR_model = AR(lag=2, phi=phi, sigma=sigma, intercept=0)

In [24]:
rt=AR_model.predict(Y,10)

In [25]:
dp.get_price(X[99,0],rt)

IndexError: index 99 is out of bounds for axis 0 with size 1